Proyecto: Predicción de Lluvia - Colombia

# LIMPIEZA, CARGA Y UNIFICACIÓN DE ARCHIVOS

## Importación de librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
import re
import os
#import plotly.express as pex
#from plotly.subplots import make_subplots
#import plotly.graph_objects as go

*Sistema de archivos*

    ./
    
    ./Proyecto/
        ./Proyecto/Proyecto_bootcamp_cuaderno.ipynb

    ./Datasets/
        ./Datasets/...... .csv
    
    ./CleanDatasets/
        ./CleanDatasets/.....  .parquet


A continuación se listan los archivos de la segunda carpeta que contiene los datasets:

    - Catálogo nacional de estaciones (Contine el listado de las estaciones)
    - Datos de estaciones IDEAM y terceros (Contiene mediciones de diversas estaciones propias y externas al IDEAM)
    - Datos hidrometeorológicos (Mediciones de temperatura)
    - Presión atmosférica  

## Conversión de Formato CSV a PARQUET

### Datos Precipitaciones

Se importa el primer documento previo a la limpieza. Se inicia con el archivo de menor tamaño. Todos los datos corresponden a precipitación y tienen la misma unidad de medida.

In [28]:
df = pd.read_csv('../Datasets/Precipitaci_n_20241016(vaupes).csv',
                 sep=',',
                 #nrows=10
                 )
df

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,42077020,240,11/01/2023 07:30:00 PM,0.2,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
1,42077020,240,11/01/2023 11:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
2,42077020,240,11/01/2023 03:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
3,42077020,240,11/01/2023 09:30:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
4,42077020,240,11/01/2023 05:40:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
...,...,...,...,...,...,...,...,...,...,...,...,...
9147,42077020,240,06/22/2024 02:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
9148,42077020,240,06/22/2024 01:30:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
9149,42077020,240,06/22/2024 01:40:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
9150,42077020,240,06/22/2024 01:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm


En la siguiente celda se imprime un pequeño resumen. Es importante resaltar que no hay datos nulos y la necesidad de cambiar el formato de cada columna por uno mas adecuado que reduzca el tamaño del dataframe.

In [29]:
size_0=df.memory_usage(deep=True).sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9152 entries, 0 to 9151
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CodigoEstacion     9152 non-null   int64  
 1   CodigoSensor       9152 non-null   int64  
 2   FechaObservacion   9152 non-null   object 
 3   ValorObservado     9152 non-null   float64
 4   NombreEstacion     9152 non-null   object 
 5   Departamento       9152 non-null   object 
 6   Municipio          9152 non-null   object 
 7   ZonaHidrografica   9152 non-null   object 
 8   Latitud            9152 non-null   float64
 9   Longitud           9152 non-null   float64
 10  DescripcionSensor  9152 non-null   object 
 11  UnidadMedida       9152 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 858.1+ KB


#### Primera exploración - Vaupes

Se importa nuevamente el archivo csv, seleccionando las columnas relevantes y eligiendo un formato adeacuado para cada columna.

In [30]:
df = pd.read_csv('../Datasets/Precipitaci_n_20241016(vaupes).csv',
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
df

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,0042077020,0240,11/01/2023 07:30:00 PM,0.2,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
1,0042077020,0240,11/01/2023 11:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
2,0042077020,0240,11/01/2023 03:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
3,0042077020,0240,11/01/2023 09:30:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
4,0042077020,0240,11/01/2023 05:40:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
...,...,...,...,...,...,...,...,...,...,...
9147,0042077020,0240,06/22/2024 02:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9148,0042077020,0240,06/22/2024 01:30:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9149,0042077020,0240,06/22/2024 01:40:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9150,0042077020,0240,06/22/2024 01:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9152 entries, 0 to 9151
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   CodigoEstacion    9152 non-null   category
 1   CodigoSensor      9152 non-null   category
 2   FechaObservacion  9152 non-null   object  
 3   ValorObservado    9152 non-null   float32 
 4   NombreEstacion    9152 non-null   category
 5   Departamento      9152 non-null   category
 6   Municipio         9152 non-null   category
 7   ZonaHidrografica  9152 non-null   category
 8   Latitud           9152 non-null   category
 9   Longitud          9152 non-null   category
dtypes: category(8), float32(1), object(1)
memory usage: 178.9+ KB


Se muestra una comparación entre el tamaño del dataframe tras el cambio de formato a las columnas

In [32]:
size_f=df.memory_usage(deep=True).sum()
print(size_f/size_0)

0.1759772734217371


**Formato de fecha**

In [33]:
df.loc[:,'FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], format='%m/%d/%Y %I:%M:%S %p')

In [34]:
df

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,0042077020,0240,2023-11-01 19:30:00,0.2,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
1,0042077020,0240,2023-11-01 23:10:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
2,0042077020,0240,2023-11-01 15:10:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
3,0042077020,0240,2023-11-01 21:30:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
4,0042077020,0240,2023-11-01 17:40:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
...,...,...,...,...,...,...,...,...,...,...
9147,0042077020,0240,2024-06-22 02:20:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9148,0042077020,0240,2024-06-22 01:30:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9149,0042077020,0240,2024-06-22 01:40:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9150,0042077020,0240,2024-06-22 01:20:00,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9152 entries, 0 to 9151
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   CodigoEstacion    9152 non-null   category
 1   CodigoSensor      9152 non-null   category
 2   FechaObservacion  9152 non-null   object  
 3   ValorObservado    9152 non-null   float32 
 4   NombreEstacion    9152 non-null   category
 5   Departamento      9152 non-null   category
 6   Municipio         9152 non-null   category
 7   ZonaHidrografica  9152 non-null   category
 8   Latitud           9152 non-null   category
 9   Longitud          9152 non-null   category
dtypes: category(8), float32(1), object(1)
memory usage: 178.9+ KB


In [39]:
df.to_parquet('../CleanDatasets/rain_vaupes.parquet')

In [69]:
namesFiles = os.listdir('../Datasets')
namesFiles = [re.findall(r'\((\w*)\)',x)[0] for x in namesFiles]
print(namesFiles)

['amazonas', 'antioquia', 'arauca', 'atlantico1', 'atlantico2', 'bogota1', 'bogota2', 'bogota3', 'bolivar1', 'bolivar2', 'boyaca', 'caldas', 'caqueta1', 'caqueta2', 'casanare', 'cauca', 'cesar', 'choco', 'choco1', 'cordoba1', 'cordoba2', 'cundinamarca', 'guainia', 'guajira', 'guaviare', 'huila', 'magdalena', 'meta', 'narino', 'narino1', 'nill', 'norte_de_santander', 'putumayo', 'quindio', 'risaralda', 'sanandres1', 'sanandres2', 'sanandres3', 'sanandres4', 'santander', 'sucre', 'tolima', 'valle_del_cauca', 'vaupes', 'vaupes1', 'vichada']


#### Todos los departamentos

In [70]:
print(namesFiles)
print(len(namesFiles))

['amazonas', 'antioquia', 'arauca', 'atlantico1', 'atlantico2', 'bogota1', 'bogota2', 'bogota3', 'bolivar1', 'bolivar2', 'boyaca', 'caldas', 'caqueta1', 'caqueta2', 'casanare', 'cauca', 'cesar', 'choco', 'choco1', 'cordoba1', 'cordoba2', 'cundinamarca', 'guainia', 'guajira', 'guaviare', 'huila', 'magdalena', 'meta', 'narino', 'narino1', 'nill', 'norte_de_santander', 'putumayo', 'quindio', 'risaralda', 'sanandres1', 'sanandres2', 'sanandres3', 'sanandres4', 'santander', 'sucre', 'tolima', 'valle_del_cauca', 'vaupes', 'vaupes1', 'vichada']
46


In [47]:
for name in namesFiles[:]:
    nameFile = '../Datasets/Precipitaci_n_20241016('+ name +').csv'
    df_dept = pd.read_csv(nameFile,
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
    df_dept['FechaObservacion'] = pd.to_datetime(df_dept['FechaObservacion'], format='%m/%d/%Y %I:%M:%S %p')
    nulls = df_dept.count(axis=1).sum()
    size = df_dept.shape[0]*df_dept.shape[1]
    if nulls-size != 0:
        print(f'{nulls-size} nulls in {name}, parquet file can not created')
    else:
        df_dept.to_parquet(f'../CleanDatasets/rain_{name}.parquet')
        print(f'Not nulls in {name}, parquet file created')


Not nulls in amazonas, parquet file created
Not nulls in antioquia, parquet file created
Not nulls in arauca, parquet file created
Not nulls in atlantico1, parquet file created
Not nulls in atlantico2, parquet file created
Not nulls in bogota1, parquet file created
Not nulls in bogota2, parquet file created
Not nulls in bogota3, parquet file created
Not nulls in bolivar1, parquet file created
Not nulls in bolivar2, parquet file created
Not nulls in boyaca, parquet file created
Not nulls in caldas, parquet file created
Not nulls in caqueta1, parquet file created
Not nulls in caqueta2, parquet file created
Not nulls in casanare, parquet file created
Not nulls in cauca, parquet file created
Not nulls in cesar, parquet file created
Not nulls in choco, parquet file created
Not nulls in choco1, parquet file created
Not nulls in cordoba1, parquet file created
Not nulls in cordoba2, parquet file created
Not nulls in cundinamarca, parquet file created
Not nulls in guainia, parquet file created


### Datos Estaciones IDEAM

In [48]:
namesFiles2 = os.listdir('../Datasets2')
namesFiles2 = [x for x in namesFiles2]
print(namesFiles2)

['Cat_logo_Nacional_de_Estaciones_del_IDEAM_20241017.csv', 'Datos_de_Estaciones_de_IDEAM_y_de_Terceros_20241017.csv', 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(0).csv', 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(1).csv', 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(2).csv', 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(3).csv', 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(4).csv', 'Presi_n_Atmosf_rica_20241017.csv']


La ruta de acceso al archivo de Datos estaciones IDEAM es f'../Datasets2/{namesFiles2[1]}'

In [49]:
df_estaciones = pd.read_csv(f'../Datasets2/{namesFiles2[1]}',
                            sep=',',
                            nrows=3
)
df_estaciones

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida,Entidad
0,2120500204,69,10/15/2024 09:04:00 PM,15.8,IDEAM PUENTE ARANDA,BOGOTÁ,BOGOTÁ D.C,ALTO MAGDALENA,4.621556,-74.104709,TEMPERATURA DEL AIRE MÁXIMA A 2 m,°C,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
1,2120500204,70,10/15/2024 09:59:00 PM,15.5,IDEAM PUENTE ARANDA,BOGOTÁ,BOGOTÁ D.C,ALTO MAGDALENA,4.621556,-74.104709,TEMPERATURA MÍNIMA DEL AIRE A 2 m,°C,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
2,2120500204,69,10/15/2024 10:25:00 PM,15.7,IDEAM PUENTE ARANDA,BOGOTÁ,BOGOTÁ D.C,ALTO MAGDALENA,4.621556,-74.104709,TEMPERATURA DEL AIRE MÁXIMA A 2 m,°C,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...


In [50]:
df_estaciones = pd.read_csv(f'../Datasets2/{namesFiles2[1]}',
                            sep=',',
                            dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category',12:'category'}
                            #nrows=3
)
df_estaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202642 entries, 0 to 202641
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   CodigoEstacion     202642 non-null  category
 1   CodigoSensor       202642 non-null  category
 2   FechaObservacion   202642 non-null  object  
 3   ValorObservado     202642 non-null  float32 
 4   NombreEstacion     202642 non-null  category
 5   Departamento       202642 non-null  category
 6   Municipio          202642 non-null  category
 7   ZonaHidrografica   202642 non-null  category
 8   Latitud            202642 non-null  category
 9   Longitud           202642 non-null  category
 10  DescripcionSensor  202642 non-null  category
 11  UnidadMedida       202642 non-null  category
 12  Entidad            202642 non-null  category
dtypes: category(11), float32(1), object(1)
memory usage: 5.3+ MB


Se evidencia que no hay valores nulos. Hacemos una conversión a formato de fecha y almacenamos el archivo en formato .parquet

In [51]:
df_estaciones['FechaObservacion'] = pd.to_datetime(df_estaciones['FechaObservacion'], format='%m/%d/%Y %I:%M:%S %p')
df_estaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202642 entries, 0 to 202641
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   CodigoEstacion     202642 non-null  category      
 1   CodigoSensor       202642 non-null  category      
 2   FechaObservacion   202642 non-null  datetime64[ns]
 3   ValorObservado     202642 non-null  float32       
 4   NombreEstacion     202642 non-null  category      
 5   Departamento       202642 non-null  category      
 6   Municipio          202642 non-null  category      
 7   ZonaHidrografica   202642 non-null  category      
 8   Latitud            202642 non-null  category      
 9   Longitud           202642 non-null  category      
 10  DescripcionSensor  202642 non-null  category      
 11  UnidadMedida       202642 non-null  category      
 12  Entidad            202642 non-null  category      
dtypes: category(11), datetime64[ns](1), float32(

In [53]:
df_estaciones.to_parquet('../CleanDatasets2/estaciones.parquet')

### Datos Presión Atmosférica

In [54]:
namesFiles2

['Cat_logo_Nacional_de_Estaciones_del_IDEAM_20241017.csv',
 'Datos_de_Estaciones_de_IDEAM_y_de_Terceros_20241017.csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(0).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(1).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(2).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(3).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(4).csv',
 'Presi_n_Atmosf_rica_20241017.csv']

La ruta de acceso a los datos de presión atmosférica es: f '../Datasets2/{namesFiles2[-1]}']

In [55]:
df_presion = pd.read_csv(
                        f'../Datasets2/{namesFiles2[-1]}',
                        sep=',',
                        nrows=3
)
df_presion

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,36015020,255,10/03/2017 06:00:00 AM,992.5,EL DIAMANTE - AUT,CASANARE,PAZ DE ARIPORO,META,5.816194,-71.419833,Presión Atmosferica (1h),HPa
1,21195190,255,02/14/2014 05:00:00 AM,785.2,PASCA - AUT,CUNDINAMARCA,PASCA,ALTO MAGDALENA,4.310111,-74.311750,Presión Atmosferica (1h),HPa
2,21015050,255,09/27/2013 06:00:00 PM,805.3,PURACE - AUT,HUILA,SAN AGUSTÍN,ALTO MAGDALENA,1.925917,-76.427556,Presión Atmosferica (1h),HPa


In [56]:
df_presion = pd.read_csv(
                        f'../Datasets2/{namesFiles2[-1]}',
                        sep=',',
                        dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category'}
)
df_presion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22611008 entries, 0 to 22611007
Data columns (total 12 columns):
 #   Column             Dtype   
---  ------             -----   
 0   CodigoEstacion     category
 1   CodigoSensor       category
 2   FechaObservacion   object  
 3   ValorObservado     float32 
 4   NombreEstacion     category
 5   Departamento       category
 6   Municipio          category
 7   ZonaHidrografica   category
 8   Latitud            category
 9   Longitud           category
 10  DescripcionSensor  category
 11  UnidadMedida       category
dtypes: category(10), float32(1), object(1)
memory usage: 582.3+ MB


In [57]:
df_presion['FechaObservacion'] = pd.to_datetime(df_presion['FechaObservacion'],format='%m/%d/%Y %I:%M:%S %p')
df_presion

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0036015020,0255,2017-10-03 06:00:00,992.500000,EL DIAMANTE - AUT,CASANARE,PAZ DE ARIPORO,META,5.816194444,-71.41983333,Presión Atmosferica (1h),HPa
1,0021195190,0255,2014-02-14 05:00:00,785.200012,PASCA - AUT,CUNDINAMARCA,PASCA,ALTO MAGDALENA,4.310111111,-74.31175,Presión Atmosferica (1h),HPa
2,0021015050,0255,2013-09-27 18:00:00,805.299988,PURACE - AUT,HUILA,SAN AGUSTÍN,ALTO MAGDALENA,1.925916667,-76.42755556,Presión Atmosferica (1h),HPa
3,0021115010,0255,2005-11-28 10:00:00,958.500000,DESIERTO TATACOA,HUILA,VILLAVIEJA,ALTO MAGDALENA,3.234,-75.168,Presión Atmosferica (1h),HPa
4,0028035060,0255,2008-04-08 04:00:00,988.700012,FEDEARROZ - AUT,CESAR,VALLEDUPAR,CESAR,10.46361111,-73.24805556,Presión Atmosferica (1h),HPa
...,...,...,...,...,...,...,...,...,...,...,...,...
22611003,0048015050,0258,2024-10-16 23:04:00,1000.500000,AEROPUERTO VASQUEZ COBO,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS,-4.193861111,-69.94091667,GPRS - PRESIÓN ATMOSFÉRICA,hPA
22611004,0048015050,0258,2024-10-16 09:08:00,1004.099976,AEROPUERTO VASQUEZ COBO,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS,-4.193861111,-69.94091667,GPRS - PRESIÓN ATMOSFÉRICA,hPA
22611005,0016015501,0258,2024-10-16 19:02:00,972.900024,AEROPUERTO CAMILO DAZA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,7.93028,-72.50917,GPRS - PRESIÓN ATMOSFÉRICA,hPA
22611006,0048015040,0255,2024-10-16 22:40:00,997.400024,PUERTO NARIÑO,AMAZONAS,PUERTO NARIÑO,AMAZONAS - DIRECTOS,-3.780305556,-70.36263889,PRESIÓN ATMOSFÉRICA,hPa


In [58]:
df_presion.count()

CodigoEstacion       22611008
CodigoSensor         22611008
FechaObservacion     22611008
ValorObservado       22611008
NombreEstacion       22611008
Departamento         22611008
Municipio            22611008
ZonaHidrografica     22611008
Latitud              22611008
Longitud             22611008
DescripcionSensor    22611008
UnidadMedida         22611008
dtype: int64

Se observa con la función .count() que el tamaño de valores no nulos es igual al índice del rango del dataframe encontrado en la celda anterior.

In [59]:
df_presion.to_parquet('../CleanDatasets2/presion.parquet')

### Datos Temperatura

Los datos de temperatura se distribuyen en 5 archivos csv distintos, todos con las mismas columnas. La ruta de acceso a los archivos es 

- '../Datasets2/Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(*)' Donde * es un valor entre 0 y 4

In [60]:
namesFiles2[2:-1]

['Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(0).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(1).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(2).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(3).csv',
 'Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241017(4).csv']

In [61]:
df_temp = pd.read_csv(
                        f'../Datasets2/{namesFiles2[2]}',
                        sep=',',
                        nrows=3
)
df_temp

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,48015050,71,01/04/2024 09:58:00 PM,21.1,AEROPUERTO VASQUEZ COBO,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS,-4.193861,-69.940917,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
1,15065180,71,01/04/2024 07:46:00 PM,22.3,AEROPUERTO ALM. PADILLA -,LA GUAJIRA,RIOHACHA,CARIBE - GUAJIRA,11.528444,-72.917722,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
2,3502500135,71,01/04/2024 03:56:00 AM,17.0,GUAYABETAL POLLO OLIMPICO,CUNDINAMARCA,GUAYABETAL,META,4.225530,-73.814810,GPRS - TEMPERATURA DEL AIRE A 2 m,°C


In [62]:
df_temp = pd.read_csv(
                        f'../Datasets2/{namesFiles2[2]}',
                        sep=',',
                        dtype= {0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category'}         
                    )
df_temp

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0048015050,0071,01/04/2024 09:58:00 PM,21.100000,AEROPUERTO VASQUEZ COBO,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS,-4.19386111,-69.94091667,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
1,0015065180,0071,01/04/2024 07:46:00 PM,22.299999,AEROPUERTO ALM. PADILLA -,LA GUAJIRA,RIOHACHA,CARIBE - GUAJIRA,11.5284444,-72.91772222,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
2,3502500135,0071,01/04/2024 03:56:00 AM,17.000000,GUAYABETAL POLLO OLIMPICO,CUNDINAMARCA,GUAYABETAL,META,4.22553,-73.81481,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
3,0016015501,0071,01/04/2024 04:30:00 PM,28.500000,AEROPUERTO CAMILO DAZA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,7.93028,-72.50917,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
4,0026125710,0071,01/04/2024 06:58:00 AM,20.900000,AEROPUERTO MATECANA,RISARALDA,PEREIRA,CAUCA,4.812675,-75.73951944,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
...,...,...,...,...,...,...,...,...,...,...,...,...
4101303,0015015050,0071,10/16/2024 10:08:00 AM,29.700001,AEROPUERTO SIMON BOLIVAR,MAGDALENA,SANTA MARTA,CARIBE - GUAJIRA,11.1146944,-74.231027778,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
4101304,0026125710,0071,10/16/2024 05:32:00 AM,18.500000,AEROPUERTO MATECANA,RISARALDA,PEREIRA,CAUCA,4.812675,-75.739519444,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
4101305,0015015050,0071,10/16/2024 07:20:00 PM,28.000000,AEROPUERTO SIMON BOLIVAR,MAGDALENA,SANTA MARTA,CARIBE - GUAJIRA,11.1146944,-74.231027778,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
4101306,0017015010,0071,10/16/2024 01:04:00 PM,26.900000,AEROPUERTO SESQUICENTENARIO,ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA...,SAN ANDRÉS,ISLAS CARIBE,12.587849,-81.701117,GPRS - TEMPERATURA DEL AIRE A 2 m,°C


In [63]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4101308 entries, 0 to 4101307
Data columns (total 12 columns):
 #   Column             Dtype   
---  ------             -----   
 0   CodigoEstacion     category
 1   CodigoSensor       category
 2   FechaObservacion   object  
 3   ValorObservado     float32 
 4   NombreEstacion     category
 5   Departamento       category
 6   Municipio          category
 7   ZonaHidrografica   category
 8   Latitud            category
 9   Longitud           category
 10  DescripcionSensor  category
 11  UnidadMedida       category
dtypes: category(10), float32(1), object(1)
memory usage: 86.1+ MB


In [64]:
df_temp['FechaObservacion'] = pd.to_datetime(df_temp['FechaObservacion'], format='%m/%d/%Y %I:%M:%S %p')
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4101308 entries, 0 to 4101307
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CodigoEstacion     category      
 1   CodigoSensor       category      
 2   FechaObservacion   datetime64[ns]
 3   ValorObservado     float32       
 4   NombreEstacion     category      
 5   Departamento       category      
 6   Municipio          category      
 7   ZonaHidrografica   category      
 8   Latitud            category      
 9   Longitud           category      
 10  DescripcionSensor  category      
 11  UnidadMedida       category      
dtypes: category(10), datetime64[ns](1), float32(1)
memory usage: 86.1 MB


In [65]:
df_temp.to_parquet('../CleanDatasets2/Temp_0.parquet')

In [66]:
index_temp = 1
for i in namesFiles2[3:-1]:
    df_temp = pd.read_csv(
                        f'../Datasets2/{i}',
                        sep=',',
                        dtype= {0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category',10:'category',11:'category'}         
                    )
    df_temp['FechaObservacion'] = pd.to_datetime(df_temp['FechaObservacion'], format='%m/%d/%Y %I:%M:%S %p')
    df_temp.to_parquet(f'../CleanDatasets2/Temp_{str(index_temp)}.parquet')
    index_temp += 1

## Unificación de archivos

Los archivos en formato parquet pueden ser consultados en la carpeta compartida de google drive: https://drive.google.com/drive/folders/1h9_VFIFKbZ8tiavxZM9Nm9koIxJxtCWf?usp=sharing

Se puede acceder a la carpeta compartida desde local instalando la versión de escritorio y buscando la ruta de los archivos.

CleanDatasets contiene los archivos parquet sobre precipitaciones

CleanDatasets2 contiene los archivos parquet sobre temperatura, presión, catálogo de estaciones.

### Datos Precipitaciones

Lista de archivos:

In [78]:
namesFiles = os.listdir(R'../CleanDatasets')
print(namesFiles)

['rain_amazonas.parquet', 'rain_antioquia.parquet', 'rain_arauca.parquet', 'rain_atlantico1.parquet', 'rain_atlantico2.parquet', 'rain_bogota1.parquet', 'rain_bogota2.parquet', 'rain_bogota3.parquet', 'rain_bolivar1.parquet', 'rain_bolivar2.parquet', 'rain_boyaca.parquet', 'rain_caldas.parquet', 'rain_caqueta1.parquet', 'rain_caqueta2.parquet', 'rain_casanare.parquet', 'rain_cauca.parquet', 'rain_cesar.parquet', 'rain_choco.parquet', 'rain_choco1.parquet', 'rain_cordoba1.parquet', 'rain_cordoba2.parquet', 'rain_cundinamarca.parquet', 'rain_guainia.parquet', 'rain_guajira.parquet', 'rain_guaviare.parquet', 'rain_huila.parquet', 'rain_magdalena.parquet', 'rain_meta.parquet', 'rain_narino.parquet', 'rain_narino1.parquet', 'rain_nill.parquet', 'rain_norte_de_santander.parquet', 'rain_putumayo.parquet', 'rain_quindio.parquet', 'rain_risaralda.parquet', 'rain_sanandres1.parquet', 'rain_sanandres2.parquet', 'rain_sanandres3.parquet', 'rain_sanandres4.parquet', 'rain_santander.parquet', 'rain_

#### Dimensionalidad de los dataset

Estamos interesados en encontrar los valores nulos dentro de los diferentes archivos, en la siguiente celda se calcula un DataFrame que detalla la siguiente información:

- File: Nombre del archivo
- Rows: Cantidad de filas del archivo
- Columns: Cantidad de columnas del archivo
- Size: Tamaño del dataframe, cantidad de celdas
- Nulls: Cantidad de valores nulos

Se observa que ningún archivo contiene valores nulos, además todos tienen la misma cantidad de columnas por tanto es posible formar un solo dataframe

In [81]:
dim = pd.DataFrame(columns=['File','Rows','Columns','Size','Nulls'])

for i in namesFiles:
    path_file = r"../CleanDatasets/" + i
    df_dept2 = pd.read_parquet(
                                path_file
                            )
    row = pd.DataFrame({'File':[i],
                        'Rows':[df_dept2.shape[0]],
                        'Columns':[df_dept2.shape[1]],
                        'Size':[df_dept2.size],
                        'Nulls':[df_dept2.size-df_dept2.count().sum()],
                        'CodigoEstacion': df_dept2['CodigoEstacion'].dtype,
                        'CodigoSensor': df_dept2['CodigoSensor'].dtype,
                        'FechaObservacion': df_dept2['FechaObservacion'].dtype,
                        'ValorObservado': df_dept2['ValorObservado'].dtype,
                        'NombreEstacion': df_dept2['NombreEstacion'].dtype,
                        'Departamento': df_dept2['Departamento'].dtype,
                        'Municipio': df_dept2['Municipio'].dtype,
                        'ZonaHidrografica': df_dept2['ZonaHidrografica'].dtype,
                        'Latitud': df_dept2['Latitud'].dtype,
                        'Longitud': df_dept2['Longitud'].dtype
                        }
                        )
    dim = pd.concat([dim,row], ignore_index=True)

dim

,File,Rows,Columns,Size,Nulls,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,rain_amazonas.parquet,706717,10,7067170,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
1,rain_antioquia.parquet,14178669,10,141786690,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
2,rain_arauca.parquet,794950,10,7949500,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
3,rain_atlantico1.parquet,1371211,10,13712110,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
4,rain_atlantico2.parquet,1156367,10,11563670,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
5,rain_bogota1.parquet,1779933,10,17799330,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
6,rain_bogota2.parquet,19408968,10,194089680,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
7,rain_bogota3.parquet,3180034,10,31800340,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
8,rain_bolivar1.parquet,1917368,10,19173680,0,category,category,datetime64[ns],float32,category,category,category,category,category,category
9,rain_bolivar2.parquet,1498134,10,14981340,0,category,category,datetime64[ns],float32,category,category,category,category,category,category


#### Coordenadas y nombres de las estaciones

Estamos interesados en recopilar la información de los diferentes archivos parquet para formar un solo dataframe sobre el cual se pudieran calcular algunas estadísticas exploratorias. 

Con el fin de evitar el consumo excesivo de memoria vamos a dividir el Dataframe de precipitaciones de manera que uno de ellos almacenará la ubicación de las diferentes estaciones de monitoreo de la precipitación y el otro almacenará los registros de la precipitación.

Adicionalmente con el objetivo de conservar la cantidad de registros de cada ubicación, calculamos la columa 'TotalEntries'. Es importante resaltar que la agrupación de todos los archivos indica que existen *199781152* filas.

In [82]:
df_part = pd.DataFrame()
for i in namesFiles:
    path_file = r"../CleanDatasets/" + i
    df_dept_aux = pd.read_parquet(
                                path_file,
                                columns=['CodigoEstacion','NombreEstacion','Departamento','Municipio','ZonaHidrografica','Latitud','Longitud']
                            )
    df_dept = df_dept_aux.groupby(['CodigoEstacion','NombreEstacion','Departamento','Municipio','ZonaHidrografica','Latitud','Longitud'], observed=True).agg({'Longitud':'count'})
    df_dept.columns = ['TotalEntries']
    df_dept = df_dept.reset_index()
    df_dept = df_dept.rename({'CodigoEstacion':'CodSta',
                            'NombreEstacion':'NameSta',
                            'Departamento':'Dept',
                            'Municipio':'City',
                            'ZonaHidrografica':'Zone',
                            'Latitud':'Lat',
                            'Longitud':'Long'
                            }, 
                            axis=1)
    df_dept = df_dept.astype({'CodSta':'str',
                            'NameSta':'str',
                            'Dept':'str',
                            'City':'str',
                            'Zone':'str',
                            'Lat':'float64',
                            'Long':'float64',
                            'TotalEntries':'int64'
                            })
    df_part = pd.concat([df_part,df_dept], ignore_index=True)


df_part.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CodSta        1440 non-null   object 
 1   NameSta       1440 non-null   object 
 2   Dept          1440 non-null   object 
 3   City          1440 non-null   object 
 4   Zone          1440 non-null   object 
 5   Lat           1440 non-null   float64
 6   Long          1440 non-null   float64
 7   TotalEntries  1440 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 90.1+ KB


En la siguiente celda imprimimos la cantidad de valores diferentes por cada columna con el objetivo de identificar incoherencias en los datos y posibles valores duplicados.
Por ejemplo, es posible que un municipio como Bogotá se encuentre escrito de diferentes formas como: 'Bogota', 'Bogotá, D.C.' ...etc pero que corresponden al mismo municipio. Esto puede suceder con los nombres de las estaciones y con demás columnas con formato category

In [83]:
df_part.nunique()

CodSta           894
NameSta         1186
Dept              46
City             505
Zone              34
Lat             1191
Long            1098
TotalEntries    1307
dtype: int64

Note que existen 894 códigos de estación pero existen 1186 nombres de estación, 1191 latitudes y 1098 longitudes. Esto indica que para algunos nombres de estación y coordenadas están escritos de manera diferente para una misma estación. Vamos a hacer un pequeño análisis para identificar estas incongruencias y evitar pérdidas de información.

In [84]:
df_part.groupby(['CodSta','NameSta','Lat','Long'], observed=True).agg({'TotalEntries':'sum'}).reset_index()

,CodSta,NameSta,Lat,Long,TotalEntries
0,0011017020,PR CHOCO: BAGADO,5.412000,-76.418000,60599
1,0011025501,CARMEN DE ATRATO - AUT,5.888719,-76.145167,499161
2,0011027030,EL SIETE,5.862000,-76.152056,1023
3,0011027030,EL SIETE - AUT,5.862000,-76.152056,179006
4,0011027070,BORAUDO,5.515000,-76.576000,409792
...,...,...,...,...,...
1370,5311500056,UNIVERSIDAD DEL PACIFICO - AUT,3.848308,-76.987017,129900
1371,5311500121,LA CUMBRE,3.645194,-76.564750,996
1372,5311500121,LA CUMBRE - AUT,3.645194,-76.564750,151366
1373,5311500149,COLEGIO VASCO NUÑEZ DE BALBOA,3.884194,-77.049381,8271


Se observa que al filtrar por Código de Estación, Nombre de Estación, Latitud y Longitud se encuentran 1375 filas lo cual contrasta con los 894 valores únicos de CodigoEstacion, se evidencia que algunas estaciones tienen el nombre de la estación o las coordenadas escritas de maneras diferentes. Empezaremos por corregir las coordenadas Latitud y Longitud:

##### Coordenadas

Aplicamos una primera aproximación donde redondeamos las coordenadas *Latitud* y *Longitud* a 3 decimales, esto se justifica en que 0.001° equivalen aproximadamente a 100m. Realizamos un conteó de los registros totales que debe coincidir con el valor inicial de *199781152*.

En esta primera celda calculamos el total de registros por cada ubicación, incluimos el nombre de la estación para consultas futuras en el procedimiento de limpieza de las coordenadas.

In [85]:
coordSta_0 = df_part.groupby(['CodSta','NameSta','Lat','Long'], observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_0[['Lat','Long']] = coordSta_0[['Lat','Long']].round(3)
print(f'Total Entries: {coordSta_0['TotalEntries'].agg('sum')}')
coordSta_0

Total Entries: 199781152


,CodSta,NameSta,Lat,Long,TotalEntries
0,0011017020,PR CHOCO: BAGADO,5.412,-76.418,60599
1,0011025501,CARMEN DE ATRATO - AUT,5.889,-76.145,499161
2,0011027030,EL SIETE,5.862,-76.152,1023
3,0011027030,EL SIETE - AUT,5.862,-76.152,179006
4,0011027070,BORAUDO,5.515,-76.576,409792
...,...,...,...,...,...
1370,5311500056,UNIVERSIDAD DEL PACIFICO - AUT,3.848,-76.987,129900
1371,5311500121,LA CUMBRE,3.645,-76.565,996
1372,5311500121,LA CUMBRE - AUT,3.645,-76.565,151366
1373,5311500149,COLEGIO VASCO NUÑEZ DE BALBOA,3.884,-77.049,8271


Al agrupar el dataframe por CodSta, Lat y Long se obtiene un total de 968 filas, de manera que existen Códigos de estación con diferentes coordenadas. Realizaremos ajustes a las coordenadas para obtener una coordenada única para cada estación. 

In [86]:
coordSta_1 = coordSta_0.groupby(['CodSta','Lat','Long'],observed=True).agg({'TotalEntries':'sum'}).reset_index()
print(f'Total Entries: {coordSta_1['TotalEntries'].agg('sum')}')
coordSta_1

Total Entries: 199781152


,CodSta,Lat,Long,TotalEntries
0,0011017020,5.412,-76.418,60599
1,0011025501,5.889,-76.145,499161
2,0011027030,5.862,-76.152,180029
3,0011027070,5.515,-76.576,409792
4,0011030010,5.375,-76.613,216178
...,...,...,...,...
963,5205500123,1.055,-77.270,89993
964,5311500056,3.848,-76.987,129900
965,5311500121,3.645,-76.565,152362
966,5311500149,3.884,-77.049,8271


Definimos una función que nos permite encontrar la distancia en Km entre dos ubicaciones en base a las coordenadas.

In [87]:
def Distance(Lat1:float,Long1:float,Lat2:float,Long2:float):
    Lat1, Long1, Lat2, Long2 = map(np.radians, [Lat1, Long1, Lat2, Long2])
    DeltaLong = Long2-Long1
    DeltaLat = Lat2-Lat1
    R = 6371   #Asumiendo un modelo esférico de la tierra  
    return round(2*R*np.arcsin(np.sqrt(np.sin(DeltaLat/2)**2 + ( np.cos(Lat1)*np.cos(Lat2)*np.sin(DeltaLong/2)**2 ))),3)

Calculamos para cada Código de Estación el valor medio de las diferentes coordenadas. Adicionalmente calculamos la columna *Distance* como la distancia de la coordenada registrada al punto coordenado medio.

In [88]:
coordSta_aux = coordSta_1.groupby(['CodSta'], observed=True).agg({'Lat':'mean','Long':'mean'}).round(3)
coordSta_aux = coordSta_aux.reset_index()
coordSta_1 = coordSta_1[['CodSta','Lat','Long','TotalEntries']].merge(coordSta_aux, how='inner', on='CodSta', suffixes=('','_mean'))
coordSta_1['Distance'] = coordSta_1.apply(lambda x: Distance(x['Lat'],x['Long'],x['Lat_mean'],x['Long_mean']), axis=1)
print(f'Total Entries: {coordSta_1['TotalEntries'].agg('sum')}')
coordSta_1

Total Entries: 199781152


,CodSta,Lat,Long,TotalEntries,Lat_mean,Long_mean,Distance
0,0011017020,5.412,-76.418,60599,5.412,-76.418,0.000
1,0011025501,5.889,-76.145,499161,5.889,-76.145,0.000
2,0011027030,5.862,-76.152,180029,5.862,-76.152,0.000
3,0011027070,5.515,-76.576,409792,5.515,-76.576,0.000
4,0011030010,5.375,-76.613,216178,5.377,-76.612,0.248
...,...,...,...,...,...,...,...
963,5205500123,1.055,-77.270,89993,1.055,-77.270,0.000
964,5311500056,3.848,-76.987,129900,3.848,-76.987,0.000
965,5311500121,3.645,-76.565,152362,3.645,-76.565,0.000
966,5311500149,3.884,-77.049,8271,3.884,-77.049,0.000


Consideramos un radio de precisión de las coordenadas de 1.5Km. Los registros de coordenadas con distancias menores a 1.5Km los reemplazamos con el valor medio de las coordenadas. El dataframe *coordSta_2* almacenará las coordenadas de las estaciones que presentan variaciones en sus coordenadas, menores a un radio de 1.5Km desde el punto medio de las coordenadas.

In [89]:
coordSta_2 = coordSta_1[coordSta_1['Distance']<=1.5].copy()
coordSta_2['Lat'] = coordSta_2['Lat_mean']
coordSta_2['Long'] = coordSta_2['Long_mean']
coordSta_2 = coordSta_2.drop(['Lat_mean','Long_mean'], axis=1)
coordSta_2 = coordSta_2.groupby(['CodSta','Lat','Long']).agg({'TotalEntries':'sum'})
coordSta_2 = coordSta_2.reset_index()
print(f'Total Entries: {coordSta_2['TotalEntries'].agg('sum')}')
coordSta_2

Total Entries: 194597200


,CodSta,Lat,Long,TotalEntries
0,0011017020,5.412,-76.418,60599
1,0011025501,5.889,-76.145,499161
2,0011027030,5.862,-76.152,180029
3,0011027070,5.515,-76.576,409792
4,0011030010,5.377,-76.612,241903
...,...,...,...,...
873,5205500123,1.055,-77.270,89993
874,5311500056,3.848,-76.987,129900
875,5311500121,3.645,-76.565,152362
876,5311500149,3.884,-77.049,8271


En la siguiente celda listamos los registros que presentan una distancia al punto medio mayor a 1.5Km. Realizamos para cada *CodSta* el cálculo de *%Entries*, esta última columna servirá como criterio para elegir una coordenada única para cada *CodSta*.

In [90]:
coordSta_3_temp = coordSta_1[coordSta_1['Distance']>1.5].copy()

coordSta_aux = coordSta_3_temp.groupby(['CodSta'], observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_3_temp = coordSta_3_temp.merge(coordSta_aux, how='inner', on='CodSta', suffixes=('','_byCodSta'))
coordSta_3_temp['%Entries'] = coordSta_3_temp['TotalEntries']/coordSta_3_temp['TotalEntries_byCodSta']
coordSta_3_temp = coordSta_3_temp.sort_values(['CodSta','%Entries'])
coordSta_3_temp = coordSta_3_temp.drop('TotalEntries_byCodSta', axis=1)
coordSta_3_temp.groupby('CodSta',observed=True).apply(lambda x: x, include_groups=False)

Lat    Long  TotalEntries  Lat_mean  Long_mean  Distance  \
CodSta                                                                       
0011175000 1    5.586 -76.650         54344     5.558    -76.642     3.237   
           0    5.529 -76.634        163543     5.558    -76.642     3.344   
0017015010 3   12.588 -81.701        327660    12.565    -81.716     3.032   
           2   12.542 -81.731        353202    12.565    -81.716     3.032   
0021206780 5    4.619 -74.195         76792     4.618    -74.256     6.762   
           4    4.617 -74.317        316876     4.618    -74.256     6.762   
0021206810 7    4.564 -74.138         70445     4.557    -74.186     5.377   
           6    4.550 -74.233        589417     4.557    -74.186     5.267   
0021208480 8    4.611 -74.178        107012     4.622    -74.164     1.976   
           9    4.633 -74.150        111513     4.622    -74.164     1.976   
0021209920 10   5.106 -73.754          4442     5.149    -73.767     4.993   
           11   5.192 -73.779        887897     5.149    -73.767     4.963   
0024015300 12   5.643 -73.518          1054     5.650    -73.531     1.636   
           13   5.656 -73.544        184056     5.650    -73.531     1.586   
0024035410 14   5.726 -72.921          1047     5.740    -72.916     1.652   
           15   5.753 -72.911        700013     5.740    -72.916     1.548   
0025025240 16   8.543 -74.627         53270     8.544    -74.585     4.620   
           17   8.544 -74.543        109118     8.544    -74.585     4.618   
0054025010 18   4.898 -76.677         46353     4.898    -76.454    24.706   
           19   4.898 -76.232        250659     4.898    -76.454    24.595   
2319500125 21   7.737 -73.049          1025     7.694    -73.114     8.612   
           20   7.650 -73.178        122441     7.694    -73.114     8.584   
2401500086 23   5.554 -73.355          1024     5.498    -73.454    12.603   
           22   5.442 -73.552        164288     5.498    -73.454    12.508   
2403500041 25   5.788 -73.052          1052     5.788    -73.067     1.659   
           24   5.788 -73.082        157867     5.788    -73.067     1.659   
2633500119 26   3.560 -76.457           848     3.722    -76.330    22.872   
           27   3.883 -76.202         97176     3.722    -76.330    22.851   
2633700150 29   4.405 -76.101          1023     4.150    -76.224    31.464   
           28   3.895 -76.347        120951     4.150    -76.224    31.466   
3206500045 30   3.444 -73.744           997     3.480    -73.742     4.009   
           31   3.516 -73.740        106547     3.480    -73.742     4.009   

               %Entries  
CodSta                   
0011175000 1   0.249414  
           0   0.750586  
0017015010 3   0.481243  
           2   0.518757  
0021206780 5   0.195068  
           4   0.804932  
0021206810 7   0.106757  
           6   0.893243  
0021208480 8   0.489701  
           9   0.510299  
0021209920 10  0.004978  
           11  0.995022  
0024015300 12  0.005694  
           13  0.994306  
0024035410 14  0.001493  
           15  0.998507  
0025025240 16  0.328041  
           17  0.671959  
0054025010 18  0.156064  
           19  0.843936  
2319500125 21  0.008302  
           20  0.991698  
2401500086 23  0.006194  
           22  0.993806  
2403500041 25  0.006620  
           24  0.993380  
2633500119 26  0.008651  
           27  0.991349  
2633700150 29  0.008387  
           28  0.991613  
3206500045 30  0.009271  
           31  0.990729

Teniendo en cuenta las observaciones, elegimos como criterio de selección *%Entries*>0.7, de manera que los registros de coordenadas anómalos serán reemplazados con los registros de coordenadas que cumplan esta condición.

In [91]:
coordSta_aux = coordSta_3_temp[coordSta_3_temp['%Entries']>0.7][['CodSta','Lat','Long']]
coordSta_3 = coordSta_3_temp.merge(coordSta_aux, how='left', on='CodSta', suffixes=('','_sel'))
coordSta_3.loc[coordSta_3['Lat_sel'].notna(),'Lat'] = coordSta_3['Lat_sel']
coordSta_3.loc[coordSta_3['Long_sel'].notna(),'Long'] = coordSta_3['Long_sel']

Los registros que cumplen la condición de tener un registro coordenado con *%Entries*>0.7 se muestran a continuación en el dataframe *coordSta4*:

In [92]:
coordSta_4 = coordSta_3[coordSta_3['Lat_sel'].notna()][coordSta_3.columns[:4]]
coordSta_aux = coordSta_4.groupby(['CodSta','Lat','Long'], observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_4 = coordSta_4.merge(coordSta_aux, how='left', on=['CodSta','Lat','Long'], suffixes=('','_adj'))
coordSta_4 = coordSta_4.drop('TotalEntries', axis=1)
coordSta_4.rename({'TotalEntries_adj':'TotalEntries'}, inplace=True, axis=1)
coordSta_4 = coordSta_4.drop_duplicates().reset_index(drop=True)
coordSta_4

,CodSta,Lat,Long,TotalEntries
0,0011175000,5.529,-76.634,217887
1,0021206780,4.617,-74.317,393668
2,0021206810,4.550,-74.233,659862
3,0021209920,5.192,-73.779,892339
4,0024015300,5.656,-73.544,185110
5,0024035410,5.753,-72.911,701060
6,0054025010,4.898,-76.232,297012
7,2319500125,7.650,-73.178,123466
8,2401500086,5.442,-73.552,165312
9,2403500041,5.788,-73.082,158919


Por otra parte, a continuación se lista los registros que no cumplen dicha condición. Para estas estaciones realizamos una busqueda manual en GoogleEarth para identificar las coordenadas correctas de las estaciones.

In [93]:
coordSta_5 = coordSta_3[coordSta_3['Lat_sel'].isna()][coordSta_3.columns[:4]]
coordSta_aux = coordSta_5.merge(coordSta_0, how='left', on=['CodSta','Lat','Long','TotalEntries'])
coordSta_aux

,CodSta,Lat,Long,TotalEntries,NameSta
0,0017015010,12.588,-81.701,327660,AEROPUERTO SESQUICENTENARIO
1,0017015010,12.542,-81.731,353202,APTO SESQUICENTENARIO TX GPRS
2,0021208480,4.611,-74.178,107012,KENNEDY
3,0021208480,4.633,-74.150,111513,KENNEDY - FOPAE
4,0025025240,8.543,-74.627,53270,NaN
5,0025025240,8.544,-74.543,109118,MAJAGUAL


Las coordenadas correctas de cada sitio son: 

    AEROPUERTO SESQUICENTENARIO  '0017015010':[12.588,-81.701]
    KENNEDY                      '0021208480':[4.633,-74.150]
    MAJAGUAL                     '0025025240':[8.543,-74.627]

In [94]:
coordSta_5 = coordSta_3[coordSta_3['Lat_sel'].isna()][coordSta_3.columns[:4]]
coordSta_aux = pd.DataFrame({'0017015010':[12.588,-81.701],
                             '0021208480':[4.633,-74.150],
                             '0025025240':[8.543,-74.627]
                             }, index=coordSta_5.columns[1:-1])
coordSta_aux = coordSta_aux.transpose().reset_index(names='CodSta')
coordSta_5 = coordSta_5.merge(coordSta_aux, how='left', on='CodSta', suffixes=('','_adj'))
coordSta_5['Lat'] = coordSta_5['Lat_adj']
coordSta_5['Long'] = coordSta_5['Long_adj']
coordSta_5 = coordSta_5.groupby(['CodSta','Lat','Long'],observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_5

,CodSta,Lat,Long,TotalEntries
0,0017015010,12.588,-81.701,680862
1,0021208480,4.633,-74.150,218525
2,0025025240,8.543,-74.627,162388


Finalmente, concatenamos los DataFrames que contienen las correcciones aplicadas en cada criterio usado:

    coordSta_2 : Radio de aproximación < 1.5Km
    coordSta_4 : Radio de aproximación > 1.5Km y registros con frecuencias relativas mayores a 0.7
    coordSta_5 : Corrección manual

Adicionalmente, realizamos una sumatoria del total de registros para cada CodSta con el fin de evaluar la integridad de la operación de limpieza. El valor inicial de registros es de *199781152*.

In [95]:
coordSta_f = pd.concat([coordSta_2,coordSta_4,coordSta_5], ignore_index=True)
print(f'Total Entries: {coordSta_f['TotalEntries'].agg('sum')}')
coordSta_f

Total Entries: 199781152


,CodSta,Lat,Long,TotalEntries
0,0011017020,5.412,-76.418,60599
1,0011025501,5.889,-76.145,499161
2,0011027030,5.862,-76.152,180029
3,0011027070,5.515,-76.576,409792
4,0011030010,5.377,-76.612,241903
...,...,...,...,...
889,2633700150,3.895,-76.347,121974
890,3206500045,3.516,-73.740,107544
891,0017015010,12.588,-81.701,680862
892,0021208480,4.633,-74.150,218525


##### Nombre Estaciones

Ahora nos interesa corregir los nombres de las estaciones, ya que se evidencia que algunos Códigos de Estación tienen asociados diferentes nombres.

In [96]:
NameSta_0 = coordSta_0.groupby(['CodSta','NameSta'],observed=True).agg({'TotalEntries':'sum'}).reset_index()
NameSta_0['SizeString'] = NameSta_0['NameSta'].apply(lambda x: len(x))
NameSta_0

,CodSta,NameSta,TotalEntries,SizeString
0,0011017020,PR CHOCO: BAGADO,60599,16
1,0011025501,CARMEN DE ATRATO - AUT,499161,23
2,0011027030,EL SIETE,1023,8
3,0011027030,EL SIETE - AUT,179006,14
4,0011027070,BORAUDO,409792,7
...,...,...,...,...
1215,5311500056,UNIVERSIDAD DEL PACIFICO - AUT,129900,30
1216,5311500121,LA CUMBRE,996,9
1217,5311500121,LA CUMBRE - AUT,151366,15
1218,5311500149,COLEGIO VASCO NUÑEZ DE BALBOA,8271,29


Filtramos inicialmente los registros con un único Nombre de estación para cada código de estación. 

In [97]:
NameSta_1 = NameSta_0[~NameSta_0[['CodSta']].duplicated(keep=False)][NameSta_0.columns[:-1]].reset_index(drop=True)
NameSta_1

,CodSta,NameSta,TotalEntries
0,0011017020,PR CHOCO: BAGADO,60599
1,0011025501,CARMEN DE ATRATO - AUT,499161
2,0011027070,BORAUDO,409792
3,0011037030,LA LOMA PUEBLO NUEVO - En Siniestro,312861
4,0011050020,PR CHOCO: BETE,130369
...,...,...,...
566,5204700100,PUENTE LOS DOLORES - AUT,98176
567,5205500123,LAS IGLESIAS - AUT,89993
568,5311500056,UNIVERSIDAD DEL PACIFICO - AUT,129900
569,5311500149,COLEGIO VASCO NUÑEZ DE BALBOA,8271


Dado que el nombre de la estación no es una variable de interés reelevante para el proyecto, elegimos como criterio para seleccionar un nombre de estación tener una probabilidad de ocurrencia mayor a 0.5

In [98]:
NameSta_2 = NameSta_0[NameSta_0[['CodSta']].duplicated(keep=False)].reset_index(drop=True)
NameSta_2['%Entries'] = NameSta_2.groupby('CodSta', observed=True)[['TotalEntries']].transform(lambda x: x/x.sum())
NameSta_2aux = NameSta_2.loc[NameSta_2['%Entries']>0.5,['CodSta','NameSta']]
NameSta_2 = NameSta_2.merge(NameSta_2aux, how='left', on='CodSta', suffixes=('','_adj'))
NameSta_2.loc[NameSta_2['NameSta_adj'].notna(),'NameSta'] = NameSta_2['NameSta_adj']
NameSta_3 = NameSta_2[NameSta_2['NameSta_adj'].notna()][NameSta_2.columns[:3]]
NameSta_3 = NameSta_3.groupby(['CodSta','NameSta'],observed=True).agg({'TotalEntries':'sum'}).reset_index()
NameSta_3

,CodSta,NameSta,TotalEntries
0,0011027030,EL SIETE - AUT,180029
1,0011030010,PR CHOCO: CERTEGUI,241903
2,0011035030,PR CHOCO: UNION PANAMERICANA,104716
3,0011045010,PR CHOCO: APTO EL CARANO,250266
4,0011047040,QUIBDO - AUT,595199
...,...,...,...
318,3505500061,MEDINA - AUT,125884
319,3519700092,MANI - AUT,131678
320,3602700101,SAN SALVADOR - AUT,157685
321,3706500109,ARAUCA TERMINAL - AUT,137085


Adicionalmente, realizamos una sumatoria del total de registros para cada CodSta con el fin de evaluar la integridad de la operación de limpieza. El valor inicial de registros es de *199781152*.

In [99]:
NameSta_f = pd.concat([NameSta_1,NameSta_3], ignore_index=True)
print(f'Total Entries: {NameSta_f['TotalEntries'].agg('sum')}')
NameSta_f

Total Entries: 199781152


,CodSta,NameSta,TotalEntries
0,0011017020,PR CHOCO: BAGADO,60599
1,0011025501,CARMEN DE ATRATO - AUT,499161
2,0011027070,BORAUDO,409792
3,0011037030,LA LOMA PUEBLO NUEVO - En Siniestro,312861
4,0011050020,PR CHOCO: BETE,130369
...,...,...,...
889,3505500061,MEDINA - AUT,125884
890,3519700092,MANI - AUT,131678
891,3602700101,SAN SALVADOR - AUT,157685
892,3706500109,ARAUCA TERMINAL - AUT,137085


Recopilamos los resultados de la limpieza de las coordenadas y los nombres de las estaciones en un solo dataframe.

In [100]:
df_part_f = NameSta_f.merge(coordSta_f, how='inner', on='CodSta', suffixes=('','_coord'))
print(f'Total entries of NameSta: {df_part_f['TotalEntries'].sum()}')
print(f'Total entries of CoordSta: {df_part_f['TotalEntries_coord'].sum()}')
df_part_f = df_part_f.drop(['TotalEntries','TotalEntries_coord'],axis=1)
df_part_f

Total entries of NameSta: 199781152
Total entries of CoordSta: 199781152


,CodSta,NameSta,Lat,Long
0,0011017020,PR CHOCO: BAGADO,5.412,-76.418
1,0011025501,CARMEN DE ATRATO - AUT,5.889,-76.145
2,0011027070,BORAUDO,5.515,-76.576
3,0011037030,LA LOMA PUEBLO NUEVO - En Siniestro,5.585,-76.753
4,0011050020,PR CHOCO: BETE,5.995,-76.780
...,...,...,...,...
889,3505500061,MEDINA - AUT,4.508,-73.348
890,3519700092,MANI - AUT,4.816,-72.309
891,3602700101,SAN SALVADOR - AUT,6.228,-71.618
892,3706500109,ARAUCA TERMINAL - AUT,7.064,-70.733


#### Municipio, Departamento y ZonaHidrografica

Realizamos un conteo de los valores únicos de cada columna.

In [101]:
Ubic_0 = df_part[['CodSta','Dept','City','Zone']].copy()
Ubic_0 = Ubic_0.drop_duplicates().reset_index(drop=True)
Ubic_0.nunique()

CodSta    894
Dept       46
City      505
Zone       34
dtype: int64

##### Departamentos

Realizamos una limpieza sobre los nombres de los departamentos, esto lo realizamos de manera manual debido a que son correcciones ortográficas.

In [102]:
Dept = Ubic_0['Dept'].drop_duplicates().reset_index(drop=True)
Dept = Dept.astype('str')
print(list(Dept))

['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLÁNTICO', 'ATLANTICO', 'BOGOTÁ', 'BOGOTA D.C.', 'BOGOTA', 'CUNDINAMARCA', 'BOLIVAR', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETA', 'CAQUETÁ', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CHOCÓ', 'CÓRDOBA', 'CORDOBA', 'GUAINÍA', 'LA GUAJIRA', 'GUAVIARE', 'HUILA', 'MAGDALENA', 'META', 'NARINO', 'NARIÑO', '<nil>', 'NORTE DE SANTANDER', 'PUTUMAYO', 'QUINDÍO', 'RISARALDA', 'SAN ANDRÉS PROVIDENCIA', 'ARCHIPIÉLAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VAUPÉS', 'VICHADA']


In [103]:
Ubic_0.replace({'Dept':r'ATL.NTICO'},{'Dept':'ATLÁNTICO'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'.*BOG.*'},{'Dept':'BOGOTÁ, D.C.'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'BOL.VAR'},{'Dept':'BOLÍVAR'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'CAQUET.*'},{'Dept':'CAQUETÁ'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'CHOC.'},{'Dept':'CHOCÓ'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'C.RDOBA'},{'Dept':'CÓRDOBA'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'.*SAN AND.*'},{'Dept':'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA'},regex=True, inplace=True)
Ubic_0.replace({'Dept':'NARINO'},{'Dept':'NARIÑO'},regex=True, inplace=True)
Ubic_0.replace({'Dept':'VAUPES'},{'Dept':'VAUPÉS'},regex=True, inplace=True)
Dept = Ubic_0['Dept'].drop_duplicates().reset_index(drop=True)
Dept = Dept.astype('str')
print(list(Dept))


['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLÁNTICO', 'BOGOTÁ, D.C.', 'CUNDINAMARCA', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'GUAINÍA', 'LA GUAJIRA', 'GUAVIARE', 'HUILA', 'MAGDALENA', 'META', 'NARIÑO', '<nil>', 'NORTE DE SANTANDER', 'PUTUMAYO', 'QUINDÍO', 'RISARALDA', 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPÉS', 'VICHADA']


Vemos en la siguiente celda que las columnas Dept, City y Zone contienen un valor nulo nil por tanto, vamos a obtener la información de las coordenadas asociadas a estos códigos de estación para recuperar los valores correctos de las columnas con estos datos nulos.

In [104]:
Ubic_D1 = Ubic_0[Ubic_0['Dept']=='<nil>']
Ubic_D1.merge(df_part_f, how='inner', on='CodSta')

,CodSta,Dept,City,Zone,NameSta,Lat,Long
0,0016027120,<nil>,<nil>,CATATUMBO,SAN JAVIER - RIO ZULIA,7.834,-72.650
1,0023157050,<nil>,<nil>,<nil>,BARRANCABERMEJA,7.060,-73.877
2,0035217080,<nil>,<nil>,<nil>,PERENCO: LA CABANA - TERMO ELECTRICA,5.438,-72.455
3,0035227020,<nil>,<nil>,<nil>,PERENCO: OROCUE PIPESCA,4.783,-71.345
4,0035237040,<nil>,<nil>,<nil>,PERENCO: TRINIDAD METEO,5.419,-71.666
5,0088112901,<nil>,<nil>,<nil>,ECI JULIO GARAVITO EST. EN PRUEBAS,0.000,0.000


Realizando una busquedo en google earth encontramos la siguiente información:
    {0016027120: {'Dept':'NORTE DE SANTANDER','City':'DURANIA', 'Zone':'CATATUMBO}}
    {0023157050: {'Dept':'SANTANDER','City':'BARRANCABERMEJA', 'Zone':'MAGDALENA MEDIO'}}
    {0035217080: {'Dept':'CASANARE','City':'YOPAL', 'Zone':'META'}}
    {0035227020: {'Dept':'CASANARE','City':'OROCUÉ', 'Zone':'META'}}
    {0035237040: {'Dept':'CASANARE','City':'TRINIDAD', 'Zone':'META'}}
    {0088112901: {'Dept':'BOGOTÁ, D.C.','City':'BOGOTÁ, D.C.', 'Zone':'MAGDALENA MEDIO'}}

In [108]:
Ubic_D2 =pd.DataFrame([{'CodSta':'0016027120','Dept':'NORTE DE SANTANDER','City':'DURANIA', 'Zone':'CATATUMBO'},
    {'CodSta':'0023157050','Dept':'SANTANDER','City':'BARRANCABERMEJA', 'Zone':'MEDIO MAGDALENA'},
    {'CodSta':'0035217080','Dept':'CASANARE','City':'YOPAL', 'Zone':'META'},
    {'CodSta':'0035227020','Dept':'CASANARE','City':'OROCUÉ', 'Zone':'META'},
    {'CodSta':'0035237040','Dept':'CASANARE','City':'TRINIDAD', 'Zone':'META'},
    {'CodSta':'0088112901','Dept':'BOGOTÁ, D.C.','City':'BOGOTÁ, D.C.', 'Zone':'MEDIO MAGDALENA'},    
    ]
)
Ubic_D2

,CodSta,Dept,City,Zone
0,0016027120,NORTE DE SANTANDER,DURANIA,CATATUMBO
1,0023157050,SANTANDER,BARRANCABERMEJA,MEDIO MAGDALENA
2,0035217080,CASANARE,YOPAL,META
3,0035227020,CASANARE,OROCUÉ,META
4,0035237040,CASANARE,TRINIDAD,META
5,0088112901,"BOGOTÁ, D.C.","BOGOTÁ, D.C.",MEDIO MAGDALENA


In [109]:
Ubic_1 = Ubic_0.merge(Ubic_D2, how='left', on='CodSta', suffixes=('','_nil'))
Ubic_1.loc[Ubic_1['Dept_nil'].notna(),'Dept'] = Ubic_1['Dept_nil']
Ubic_1.loc[Ubic_1['City_nil'].notna(),'City'] = Ubic_1['City_nil']
Ubic_1.loc[Ubic_1['Zone_nil'].notna(),'Zone'] = Ubic_1['Zone_nil']
Ubic_1 = Ubic_1[Ubic_1.columns[:-3]]
Ubic_1

,CodSta,Dept,City,Zone
0,0044197020,AMAZONAS,LA PEDRERA,CAQUETÁ
1,0048015010,AMAZONAS,LETICIA,<nil>
2,0048015040,AMAZONAS,PUERTO NARIÑO,AMAZONAS - DIRECTOS
3,0048015050,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS
4,0011057020,ANTIOQUIA,VIGIA DEL FUERTE,ATRATO - DARIÉN
...,...,...,...,...
1032,0042077020,VAUPÉS,MITÚ,VAUPES
1033,0042077020,VAUPÉS,MITÚ,VAUPES
1034,0035260050,VICHADA,LA PRIMAVERA,META
1035,3526500201,VICHADA,LA PRIMAVERA,META


In [110]:
print(Ubic_1['Dept'].unique())

['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLÁNTICO' 'BOGOTÁ, D.C.' 'CUNDINAMARCA'
 'BOLÍVAR' 'BOYACÁ' 'CALDAS' 'CAQUETÁ' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCÓ'
 'CÓRDOBA' 'GUAINÍA' 'LA GUAJIRA' 'GUAVIARE' 'HUILA' 'MAGDALENA' 'META'
 'NARIÑO' 'NORTE DE SANTANDER' 'SANTANDER' 'PUTUMAYO' 'QUINDÍO'
 'RISARALDA' 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA'
 'SUCRE' 'TOLIMA' 'VALLE DEL CAUCA' 'VAUPÉS' 'VICHADA']


##### Municipios

Nos interesamos inicialmente en encontrar aquellas estaciones que tienen asociados más de un municipio.

In [111]:
Ubic_M1 = Ubic_1[['CodSta','City']].drop_duplicates()
Ubic_M1 = Ubic_M1[Ubic_M1['CodSta'].duplicated(keep=False)].reset_index(drop=True)
Ubic_M1 = Ubic_M1.merge(coordSta_f[['CodSta','Lat','Long']], how='inner', on='CodSta', suffixes=('','_adj'))
Ubic_M1 = Ubic_M1.sort_values(['CodSta','City']).reset_index(drop=True)
Ubic_M1 = Ubic_M1.groupby(['CodSta','Lat','Long'], observed=True)['City'].agg(list).reset_index()
Ubic_M1['Count'] = Ubic_M1['City'].apply(lambda x: len(x))
print(f'Maximum number of different cities by CodSta: {Ubic_M1['Count'].max()}')
Ubic_M1
#184

Maximum number of different cities by CodSta: 3


,CodSta,Lat,Long,City,Count
0,0011175000,5.529,-76.634,"[CÉRTEGUI, EL ATRATO]",2
1,0017025020,13.360,-81.358,"[SAN ANDRES Y PROVIDENCIA, SAN ANDRES Y PROV...",2
2,0021201200,4.343,-74.184,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
3,0021201580,4.446,-74.155,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
4,0021201980,4.579,-74.053,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
...,...,...,...,...,...
74,2120700038,4.562,-74.149,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
75,2120700154,4.553,-74.108,"[BOGOTA D.C, BOGOTÁ D.C]",2
76,2401500086,5.442,-73.552,"[SAMACÁ, TUNJA]",2
77,2633500119,3.883,-76.202,"[BUGA, PALMIRA]",2


In [112]:
Ubic_M1['City'].drop_duplicates().reset_index(drop=True)

0                                 [CÉRTEGUI, EL ATRATO]
1     [SAN ANDRES Y  PROVIDENCIA, SAN ANDRES Y  PROV...
2                 [BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]
3                              [BOGOTA D.C, BOGOTÁ D.C]
4                             [BOGOTA, D.C, BOGOTÁ D.C]
5                           [ARMERO, ARMERO (GUAYABAL)]
6                                 [CHAPARRAL, PLANADAS]
7                                      [ACHÍ, MAJAGUAL]
8                               [ANSERMANUEVO, ARGELIA]
9                                    [TÁMESIS, VENECIA]
10                      [MALLAMA, MALLAMA (PIEDRANCHA)]
11                            [PATÍA, PATÍA (EL BORDO)]
12                        [NÓVITA, SAN JOSÉ DEL PALMAR]
13                                   [RIO IRO, RIO IRÓ]
14               [ALTO BAUDÓ, ALTO BAUDÓ (PIE DE PATO)]
15                 [BAHÍA SOLANO, BAHÍA SOLANO (MUTIS)]
16                            [BOGOTA D.C, BOGOTA, D.C]
17                                      [SAMACÁ,

Se podría realizar un reemplazo inicial en base a las siguientes condiciones:

    - Eliminar caracteres dentro de paréntesis
    - Hacer correcciones ortográficas sobre: 'San Andrés, providencia y santa catalina', 'Bogotá, D.C.', y 'Rio Iró' 

Los cambios los aplicaremos sobre el dataframe Ubic_1

In [113]:
Ubic_1.replace({'City':r'.\(.*\)'},{'City':''},regex=True, inplace=True)
Ubic_1.replace({'City':r'.*BOG.*'},{'City':'BOGOTÁ, D.C.'},regex=True, inplace=True)
Ubic_1.replace({'City':r'.*SAN AND.*'},{'City':'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA'},regex=True, inplace=True)
Ubic_1.replace({'City':r'RIO IR.*'},{'City':'RIO IRÓ'},regex=True, inplace=True)

De nuevo aplicamos el filtrado anterior para listar las estaciones que presentan mas de un municipio en los registros. Se evidencian 9 estaciones con este tipo de diferencias.

In [114]:
Ubic_M2 = Ubic_1[['CodSta','City']].drop_duplicates()
Ubic_M2 = Ubic_M2[Ubic_M2['CodSta'].duplicated(keep=False)].reset_index(drop=True)
Ubic_M2 = Ubic_M2.merge(coordSta_f[['CodSta','Lat','Long']], how='inner', on='CodSta', suffixes=('','_adj'))
Ubic_M2 = Ubic_M2.sort_values(['CodSta','City']).reset_index(drop=True)
Ubic_M2 = Ubic_M2.groupby(['CodSta','Lat','Long'], observed=True)['City'].agg(list).reset_index()
Ubic_M2['Count'] = Ubic_M2['City'].apply(lambda x: len(x))
print(f'Maximum number of different cities by CodSta: {Ubic_M2['Count'].max()}')
Ubic_M2

Maximum number of different cities by CodSta: 2


,CodSta,Lat,Long,City,Count
0,0011175000,5.529,-76.634,"[CÉRTEGUI, EL ATRATO]",2
1,0022045502,3.950,-75.667,"[CHAPARRAL, PLANADAS]",2
2,0025025240,8.543,-74.627,"[ACHÍ, MAJAGUAL]",2
3,0026115090,4.776,-76.144,"[ANSERMANUEVO, ARGELIA]",2
4,0026205501,5.626,-75.705,"[TÁMESIS, VENECIA]",2
5,0054025010,4.898,-76.232,"[NÓVITA, SAN JOSÉ DEL PALMAR]",2
6,2401500086,5.442,-73.552,"[SAMACÁ, TUNJA]",2
7,2633500119,3.883,-76.202,"[BUGA, PALMIRA]",2
8,2633700150,3.895,-76.347,"[BUGA, ZARZAL]",2


Para resolver este conflicto, realizamos una busqueda en Google Earth para encontrar el municipio relacionado con las coordenadas de la estación. Las correcciones son las siguientes:

    - 0011175000 EL ATRATO
    - 0022045502 CHAPARRAL
    - 0025025240 MAJAGUAL
    - 0026115090 ANSERMANUEVO
    - 0026205501 TÁMESIS
    - 0054025010 SAN JOSÉ DEL PALMAR
    - 2401500086 SAMACÁ
    - 2633500119 BUGA
    - 2633700150 BUGA

In [115]:
Ubic_1.loc[Ubic_1['CodSta']=='0011175000','City'] = 'EL ATRATO'
Ubic_1.loc[Ubic_1['CodSta']=='0022045502','City'] = 'CHAPARRAL'
Ubic_1.loc[Ubic_1['CodSta']=='0025025240','City'] = 'MAJAGUAL'
Ubic_1.loc[Ubic_1['CodSta']=='0026115090','City'] = 'ANSERMANUEVO'
Ubic_1.loc[Ubic_1['CodSta']=='0026205501','City'] = 'TÁMESIS'
Ubic_1.loc[Ubic_1['CodSta']=='0054025010','City'] = 'SAN JOSÉ DEL PALMAR'
Ubic_1.loc[Ubic_1['CodSta']=='2401500086','City'] = 'SAMACÁ'
Ubic_1.loc[Ubic_1['CodSta']=='2633500119','City'] = 'BUGA'
Ubic_1.loc[Ubic_1['CodSta']=='2633700150','City'] = 'BUGA'
Ubic_1[['CodSta','City']].drop_duplicates().reset_index(drop=True)

,CodSta,City
0,0044197020,LA PEDRERA
1,0048015010,LETICIA
2,0048015040,PUERTO NARIÑO
3,0048015050,LETICIA
4,0011057020,VIGIA DEL FUERTE
...,...,...
889,5311700154,BUENAVENTURA
890,5311500149,BUENAVENTURA
891,0042077020,MITÚ
892,0035260050,LA PRIMAVERA


Con las modificaciones realizadas se encuentra que cada estación ahora queda asociada a un municipio.

##### Zonas hidrográficas

En la siguiente celda listamos las zonas hidrográficas. Sin embargo, dichas delimitaciones no representan adecuadamente las zonas hidrográficas nacionales, ya que en los registros no parece existir un criterio claro para delimitar dichas zonas. Por esta razón vamos a excluir esta columna del análisis.

In [116]:
Ubic_1['Zone'].drop_duplicates().reset_index(drop=True).sort_values().values[:]

array(['<nil>', 'ALTO MAGDALENA', 'AMAZONAS - DIRECTOS', 'ARAUCA',
       'ATRATO - DARIÉN', 'BAJO MAGDALENA',
       'BAJO MAGDALENA- CAUCA -SAN JORGE', 'BAUDÓ - DIRECTOS PACIFICO',
       'CAGUÁN', 'CAQUETÁ', 'CARIBE - GUAJIRA', 'CARIBE - LITORAL',
       'CASANARE', 'CATATUMBO', 'CAUCA', 'CESAR', 'GUAVIARE', 'INÍRIDA',
       'ISLAS CARIBE', 'MEDIO MAGDALENA', 'META', 'MIRA', 'NECHÍ',
       'ORINOCO', 'PACÍFICO - DIRECTOS', 'PATÍA', 'PUTUMAYO', 'SALDAÑA',
       'SAN JUÁN', 'SINÚ', 'SOGAMOSO', 'TAPAJE - DAGUA - DIRECTOS',
       'VAUPES', 'VICHADA'], dtype=object)

In [117]:
Ubic_2 = Ubic_1.drop('Zone', axis=1).drop_duplicates().reset_index(drop=True)
Ubic_2


,CodSta,Dept,City
0,0044197020,AMAZONAS,LA PEDRERA
1,0048015010,AMAZONAS,LETICIA
2,0048015040,AMAZONAS,PUERTO NARIÑO
3,0048015050,AMAZONAS,LETICIA
4,0011057020,ANTIOQUIA,VIGIA DEL FUERTE
...,...,...,...
891,5311700154,VALLE DEL CAUCA,BUENAVENTURA
892,5311500149,VALLE DEL CAUCA,BUENAVENTURA
893,0042077020,VAUPÉS,MITÚ
894,0035260050,VICHADA,LA PRIMAVERA


##### Estaciones registradas con diferentes departamentos

A continuación listamos un pequeño grupo de registros que presentan incoherencias en la columna Dept. Para corregir las incoherencias vamos a realizar una busqueda en internet para determinar el departamento correcto de cada municipio.

In [118]:
Ubic_2[Ubic_2['CodSta'].duplicated(keep=False)].sort_values('CodSta').reset_index(drop=True)

,CodSta,Dept,City
0,0025025240,BOLÍVAR,MAJAGUAL
1,0025025240,SUCRE,MAJAGUAL
2,2120500204,"BOGOTÁ, D.C.","BOGOTÁ, D.C."
3,2120500204,CUNDINAMARCA,"BOGOTÁ, D.C."


Los departamentos correctos de las estaciones son:

    - 0025025240 : SUCRE
    - 2120500204 : BOGOTÁ, D.C.

In [119]:
Ubic_2.loc[Ubic_2['CodSta']=='0025025240','Dept'] = 'SUCRE'
Ubic_2.loc[Ubic_2['CodSta']=='2120500204','Dept'] = 'BOGOTÁ, D.C.'
Ubic_f = Ubic_2.drop_duplicates().reset_index(drop=True)
Ubic_f

,CodSta,Dept,City
0,0044197020,AMAZONAS,LA PEDRERA
1,0048015010,AMAZONAS,LETICIA
2,0048015040,AMAZONAS,PUERTO NARIÑO
3,0048015050,AMAZONAS,LETICIA
4,0011057020,ANTIOQUIA,VIGIA DEL FUERTE
...,...,...,...
889,5311700154,VALLE DEL CAUCA,BUENAVENTURA
890,5311500149,VALLE DEL CAUCA,BUENAVENTURA
891,0042077020,VAUPÉS,MITÚ
892,0035260050,VICHADA,LA PRIMAVERA


Finalmente recopilamos los dos dataframes de limpieza de coordenadas y de ubicaciones de las 894 estaciones descritas en el dataset inicial. Adicionalmente, asignamos el tipo de dato category a las columnas: 'Dept','City'

In [120]:
dfSta = df_part_f.merge(Ubic_f, how='inner', on='CodSta').sort_values(['Dept','City'])
dfSta[['Dept','City']] = dfSta[['Dept','City']].astype('category')
dfSta = dfSta.sort_values('CodSta').reset_index(drop=True)
dfSta['CodSta'] = dfSta['CodSta'].astype('uint64')
dfSta = dfSta.rename_axis('Sta').reset_index()
dfSta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Sta      894 non-null    int64   
 1   CodSta   894 non-null    uint64  
 2   NameSta  894 non-null    object  
 3   Lat      894 non-null    float64 
 4   Long     894 non-null    float64 
 5   Dept     894 non-null    category
 6   City     894 non-null    category
dtypes: category(2), float64(2), int64(1), object(1), uint64(1)
memory usage: 59.0+ KB


In [122]:
path_file = r"..\stations.parquet"
dfSta.to_parquet(path = path_file)

#### Registros de precipitaciones

Teniendo en cuenta que contamos con un dataframe 'dfSta' que define claramente las coordenadas y ubicación de cada estación. Conviene obtener de manera similar un solo archivo que recopile los registros de las precipitaciones de todas las estaciones. Para conseguir esto, vamos a concatenar los diferentes archivos parquet y vamos a almacenar en un dataframe solo las columnas correspondientes al código de la estación, el código del sensor, la fecha de observación y el valor de la medida.

In [3]:
namesFiles = os.listdir(r'../CleanDatasets')
print(namesFiles)

['rain_amazonas.parquet', 'rain_antioquia.parquet', 'rain_arauca.parquet', 'rain_atlantico1.parquet', 'rain_atlantico2.parquet', 'rain_bogota1.parquet', 'rain_bogota2.parquet', 'rain_bogota3.parquet', 'rain_bolivar1.parquet', 'rain_bolivar2.parquet', 'rain_boyaca.parquet', 'rain_caldas.parquet', 'rain_caqueta1.parquet', 'rain_caqueta2.parquet', 'rain_casanare.parquet', 'rain_cauca.parquet', 'rain_cesar.parquet', 'rain_choco.parquet', 'rain_choco1.parquet', 'rain_cordoba1.parquet', 'rain_cordoba2.parquet', 'rain_cundinamarca.parquet', 'rain_guainia.parquet', 'rain_guajira.parquet', 'rain_guaviare.parquet', 'rain_huila.parquet', 'rain_magdalena.parquet', 'rain_meta.parquet', 'rain_narino.parquet', 'rain_narino1.parquet', 'rain_nill.parquet', 'rain_norte_de_santander.parquet', 'rain_putumayo.parquet', 'rain_quindio.parquet', 'rain_risaralda.parquet', 'rain_sanandres1.parquet', 'rain_sanandres2.parquet', 'rain_sanandres3.parquet', 'rain_sanandres4.parquet', 'rain_santander.parquet', 'rain_

En la siguiente celda importamos y concatenamos los diferentes archivos parquet, adicionalmente cambiamos las unidades de la columna 'Rain' a ${\mu m}$

In [4]:
df_rain = pd.DataFrame()
for i in namesFiles:
    path_file = r'../CleanDatasets/' + i
    df_rain_aux = pd.read_parquet(
                                path_file,
                                columns=['CodigoEstacion','CodigoSensor','FechaObservacion','ValorObservado']
                            )
    df_rain_aux = df_rain_aux.rename({'CodigoEstacion':'CodSta',       
                            'CodigoSensor':'CodSen',   
                            'FechaObservacion':'Date',
                            'ValorObservado':'Rain'
                            }, 
                            axis=1)
    df_rain_aux['Rain'] = df_rain_aux['Rain']*1000
    df_rain_aux['Date'] = pd.to_datetime(df_rain_aux['Date'])
    df_rain_aux['CodSta'] = df_rain_aux['CodSta'].astype('uint64')
    df_rain_aux['CodSen'] = df_rain_aux['CodSen'].astype('uint16')
    df_rain_aux['Rain'] = df_rain_aux['Rain'].astype('uint16')
    df_rain = pd.concat([df_rain,df_rain_aux], ignore_index=True)


df_rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199781152 entries, 0 to 199781151
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   CodSta  uint64        
 1   CodSen  uint16        
 2   Date    datetime64[ns]
 3   Rain    uint16        
dtypes: datetime64[ns](1), uint16(2), uint64(1)
memory usage: 3.7 GB


Ajustamos algunos tipos de dato con el objetivo de reducir el tamaño del dataframe y evitar un consumo excesivo de memoria. Sin embargo, es posible realizar un ajuste mas minucioso a cada columna para lograr reducir aún mas el tamaño.

   - La columna CodSta describe 894 estaciones, por tanto podríamos emplear una codificación diferente para optimizar el uso de memoria.
      - Usaremos el índice del dataframe que almacena la ubicación de las estaciones en lugar del código de la estación y le llamaremos a dicha columna Sta.
   - La columna CodSen describe solamente 2 valores diferentes a lo largo del dataframe, de manera que se puede cambiar de manera similar la codificación.
      - Para el Código de sensor 240 usaremos 0 y para el Código de sensor 257 usaremos 1.

In [5]:
path_file = r'../stations.parquet' 
dfSta = pd.read_parquet(path_file)
dfSta_cod = dfSta['CodSta'].reset_index()
dfSta_cod.rename(columns={'index':'Sta'}, inplace=True)

df_rain = df_rain.merge(dfSta_cod, how='left', on='CodSta')
df_rain = df_rain.drop('CodSta', axis=1)

df_rain.loc[df_rain['CodSen']==240,'CodSen'] = 0
df_rain.loc[df_rain['CodSen']==257,'CodSen'] = 1
df_rain

,CodSen,Date,Rain,Sta
0,0,2019-06-10 03:10:00,0,659
1,0,2017-11-12 20:30:00,0,650
2,0,2017-04-26 09:10:00,0,658
3,0,2019-03-03 06:50:00,0,658
4,0,2019-08-03 23:00:00,0,658
...,...,...,...,...
199781147,0,2024-10-15 06:20:00,0,879
199781148,0,2024-10-15 22:20:00,0,879
199781149,0,2024-10-15 10:50:00,0,879
199781150,0,2024-10-15 18:50:00,0,879


In [6]:
df_rain['CodSen'] = df_rain['CodSen'].astype('uint8')
df_rain['Sta'] = df_rain['Sta'].astype('uint16')
df_rain = df_rain.reindex(['Sta','CodSen','Date','Rain'], axis=1)
df_rain = df_rain.sort_values(['Sta','Date'])
df_rain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199781152 entries, 86050022 to 196782247
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   Sta     uint16        
 1   CodSen  uint8         
 2   Date    datetime64[ns]
 3   Rain    uint16        
dtypes: datetime64[ns](1), uint16(2), uint8(1)
memory usage: 3.9 GB


In [7]:
df_rain.to_parquet(r"..\rain_version1.parquet")

##### Separación de las observaciones de acuerdo al código de sensor

Teniendo en cuenta que las mediciones de algunas estaciones fueron tomadas simultaneamente con dos tipos de sensor, dividimos el dataframe en 2:

In [3]:
df_rain = pd.read_parquet(r"..\rain_version1.parquet")
df_rain

,Sta,CodSen,Date,Rain
86050022,0,0,2018-05-04 12:00:00,0
85276804,0,0,2018-05-04 12:10:00,0
85334374,0,0,2018-05-04 12:20:00,0
86177805,0,0,2018-05-04 12:30:00,0
84247879,0,0,2018-05-04 12:40:00,0
...,...,...,...,...
196783612,893,0,2020-08-27 11:20:00,0
196781914,893,0,2020-08-27 11:30:00,0
196783171,893,0,2020-08-27 11:40:00,0
196785212,893,0,2020-08-27 11:50:00,0


###### Sensor 0

In [4]:
df_rain0 = df_rain[df_rain['CodSen']==0]
df_rain0

,Sta,CodSen,Date,Rain
86050022,0,0,2018-05-04 12:00:00,0
85276804,0,0,2018-05-04 12:10:00,0
85334374,0,0,2018-05-04 12:20:00,0
86177805,0,0,2018-05-04 12:30:00,0
84247879,0,0,2018-05-04 12:40:00,0
...,...,...,...,...
196783612,893,0,2020-08-27 11:20:00,0
196781914,893,0,2020-08-27 11:30:00,0
196783171,893,0,2020-08-27 11:40:00,0
196785212,893,0,2020-08-27 11:50:00,0


En la siguiente celda agrupamos los registros de cada estación según el momento en el que se registró la información. Adicionalmente calculamos los valores máximo y mínimo con el objetivo de verificar que todos los registros correspondientes a un mismo perido de tiempo para una estación son iguales.

In [ ]:
RainAux0 = df_rain0.groupby(['Sta','Date'], observed=True).agg(
                                                                        Rain_min = ('Rain','min'),
                                                                        Rain_max = ('Rain','max')
                                                                        ).reset_index()

RainAux0

All duplicate records are identical: 0            True
1            True
2            True
3            True
4            True
             ... 
190977000    True
190977001    True
190977002    True
190977003    True
190977004    True
Length: 190977005, dtype: bool


In [12]:
print(f'All duplicate records are identical: {(RainAux0['Rain_min']==RainAux0['Rain_max']).all()}')

All duplicate records are identical: True


En la siguiente celda eliminamos los registros duplicados y adicionamos una columna que calcula la separación temporal de los diferentes registros.

In [13]:
Rain = df_rain0.drop_duplicates().reset_index(drop=True)
Diff = Rain.groupby('Sta', observed=True).agg({'Date':'diff'})
Rain = Rain.merge(Diff, how='left', left_index=True, right_index=True, suffixes=('','_diff'))
Rain

,Sta,CodSen,Date,Rain,Date_diff
0,0,0,2018-05-04 12:00:00,0,NaT
1,0,0,2018-05-04 12:10:00,0,0 days 00:10:00
2,0,0,2018-05-04 12:20:00,0,0 days 00:10:00
3,0,0,2018-05-04 12:30:00,0,0 days 00:10:00
4,0,0,2018-05-04 12:40:00,0,0 days 00:10:00
...,...,...,...,...,...
190977000,893,0,2020-08-27 11:20:00,0,0 days 00:10:00
190977001,893,0,2020-08-27 11:30:00,0,0 days 00:10:00
190977002,893,0,2020-08-27 11:40:00,0,0 days 00:10:00
190977003,893,0,2020-08-27 11:50:00,0,0 days 00:10:00


Debido al gran consumo de memoria de los dataframes anteriores, guardamos los cambios aplicados hasta el momento.

In [14]:
Rain.to_parquet(r"..\rain0_version2.parquet")

###### Sensor 1

In [15]:
df_rain1 = df_rain[df_rain['CodSen']==1]
df_rain1

,Sta,CodSen,Date,Rain
90409529,29,1,2023-09-12 00:00:00,0
90399836,29,1,2023-09-12 00:02:00,0
90395972,29,1,2023-09-12 00:04:00,0
90400582,29,1,2023-09-12 00:06:00,0
90392445,29,1,2023-09-12 00:08:00,0
...,...,...,...,...
115071385,870,1,2024-04-17 22:30:00,0
115073698,870,1,2024-04-17 22:32:00,0
115071070,870,1,2024-04-17 22:34:00,0
115071663,870,1,2024-04-17 22:36:00,0


En la siguiente celda agrupamos los registros de cada estación según el momento en el que se registró la información. Adicionalmente calculamos los valores máximo y mínimo con el objetivo de verificar que todos los registros correspondientes a un mismo perido de tiempo para una estación son iguales.

In [16]:
RainAux1 = df_rain1.groupby(['Sta','Date'], observed=True).agg(
                                                                        Rain_min = ('Rain','min'),
                                                                        Rain_max = ('Rain','max')
                                                                        ).reset_index()
print(f'All duplicate records are identical: {(RainAux0['Rain_min']==RainAux0['Rain_max']).all()}')

All duplicate records are identical: True


En la siguiente celda eliminamos los registros duplicados y adicionamos una columna que calcula la separación temporal de los diferentes registros.

In [17]:
Rain = df_rain1.drop_duplicates().reset_index(drop=True)
Diff = Rain.groupby('Sta', observed=True).agg({'Date':'diff'})
Rain = Rain.merge(Diff, how='left', left_index=True, right_index=True, suffixes=('','_diff'))
Rain

,Sta,CodSen,Date,Rain,Date_diff
0,29,1,2023-09-12 00:00:00,0,NaT
1,29,1,2023-09-12 00:02:00,0,0 days 00:02:00
2,29,1,2023-09-12 00:04:00,0,0 days 00:02:00
3,29,1,2023-09-12 00:06:00,0,0 days 00:02:00
4,29,1,2023-09-12 00:08:00,0,0 days 00:02:00
...,...,...,...,...,...
4109126,870,1,2024-04-17 22:30:00,0,0 days 00:02:00
4109127,870,1,2024-04-17 22:32:00,0,0 days 00:02:00
4109128,870,1,2024-04-17 22:34:00,0,0 days 00:02:00
4109129,870,1,2024-04-17 22:36:00,0,0 days 00:02:00


Debido al gran consumo de memoria de los dataframes anteriores, guardamos los cambios aplicados hasta el momento.

In [18]:
Rain.to_parquet(r"..\rain1_version2.parquet")

##### Definición del periodo de observación

En la siguiente celda observamos que no existe una continuidad en los periodos de tiempo registrados. Adicionalmente, los tiempos de muestreo no son iguales en todas las estaciones. Por tanto, realizaremos una exploración sobre los registros con el fín recuperar los registros perdidos y obtener un registro de las precipitaciones acumuladas en periodos de 6 horas.

In [198]:
Rain = pd.read_parquet(r"..\rain0_version2.parquet")
# Rain.groupby(['Sta','Date_diff'], observed=False).agg({'Date_diff':list})

Inicialmente realizaremos la exploración y agrupación de las observaciones en periodos de tiempo mas grandes y homogeneos para una sola estación:

In [263]:
Rain868 = Rain[Rain['Sta']==379]
Rain868['Date_diff'].value_counts()/Rain868['Date_diff'].value_counts().sum()

Date_diff
0 days 00:10:00     0.988182
0 days 01:10:00     0.003809
0 days 00:20:00     0.003435
0 days 00:40:00     0.001203
0 days 02:10:00     0.000811
                      ...   
4 days 03:00:00     0.000004
0 days 06:20:00     0.000004
16 days 18:00:00    0.000004
0 days 05:40:00     0.000004
3 days 13:10:00     0.000004
Name: count, Length: 100, dtype: float64

En la estación 868 observamos que el 98% de los registros fueron tomados con una separación temporal de 10 minutos, por tanto asumimos que este tiempo corresponde al periodo de cada una de las observaciones. 

In [264]:
Rain868 = Rain868.rename(columns={'Date':'T_f'})
Rain868['T_0'] = Rain868['T_f']- pd.Timedelta('10 minutes')
Rain868 = Rain868[['Sta','Rain','T_0','T_f']]
Rain868['Gaps'] = (Rain868['T_0'].shift(-1) - Rain868['T_f'])/pd.Timedelta('10 minutes')
Rain868['Gaps<3'] = Rain868.apply(lambda x: True if (x['Gaps']<=3 and np.mod(x['Gaps'],1)==0) else False, axis=1)
Rain868['Near'] = (Rain868['Gaps']!=0)|(Rain868['Gaps'].shift(+1)!=0)
Rain868 = Rain868.reset_index(drop=True).dropna()
Rain868

,Sta,Rain,T_0,T_f,Gaps,Gaps<3,Near
0,379,0,2017-01-18 21:00:00,2017-01-18 21:10:00,0.0,True,True
1,379,0,2017-01-18 21:10:00,2017-01-18 21:20:00,0.0,True,False
2,379,0,2017-01-18 21:20:00,2017-01-18 21:30:00,0.0,True,False
3,379,0,2017-01-18 21:30:00,2017-01-18 21:40:00,0.0,True,False
4,379,0,2017-01-18 21:40:00,2017-01-18 21:50:00,0.0,True,False
...,...,...,...,...,...,...,...
270167,379,0,2024-10-15 22:50:00,2024-10-15 23:00:00,0.0,True,False
270168,379,0,2024-10-15 23:00:00,2024-10-15 23:10:00,0.0,True,False
270169,379,0,2024-10-15 23:10:00,2024-10-15 23:20:00,0.0,True,False
270170,379,0,2024-10-15 23:20:00,2024-10-15 23:30:00,0.0,True,False


Es importante revisar que todos los periodos faltantes son un múltiplo 'entero' del intervalo mas común del dataframe

In [265]:
Rain868[np.mod(Rain868['Gaps'],1)!=0]

,Sta,Rain,T_0,T_f,Gaps,Gaps<3,Near


En la siguiente celda insertamos los periodos de tiempo faltantes y completamos los valores de precipitación solo para intervalos de tiempo menores o iguales a 30 minutos.

In [271]:
Rain_aux = Rain868.loc[Rain868.index.repeat(Rain868['Gaps']+1)].reset_index(drop=True)
Rain_aux['oldInd'] = np.repeat(Rain868.index, Rain868['Gaps']+1)
Rain_aux['old'] = Rain_aux.groupby('oldInd').cumcount() == 0
Rain_aux['Rain'] = np.where(Rain_aux['old'], Rain_aux['Rain'], np.nan)
Rain_aux.loc[Rain_aux['old']==False,['T_0','T_f']] = pd.NaT


Rain_aux.loc[(Rain_aux['Near']==True)&(Rain_aux['Gaps<3']==True),'Rain'] = Rain_aux['Rain'].interpolate(method='linear').round(0)
Rain_aux['T_0'] = Rain_aux['T_0'].interpolate(method='linear')
Rain_aux['T_f'] = Rain_aux['T_f'].interpolate(method='linear')

En la siguiente celda mostramos algunas filas de interés donde se puede evidenciar el llenado de valores faltantes. Por ejemplo, se completan 2 registros de 10min entre los tiempos 2019-12-14 19:30:00 y 2019-12-14 19:50:00

In [272]:
Rain_aux[(Rain_aux['Gaps']==2)&(Rain_aux['Rain']!=0)]
Rain_aux.loc[152629:152641]

,Sta,Rain,T_0,T_f,Gaps,Gaps<3,Near,oldInd,old
152629,379,0.0,2019-12-14 19:10:00,2019-12-14 19:20:00,0.0,True,False,69246,True
152630,379,0.0,2019-12-14 19:20:00,2019-12-14 19:30:00,2.0,True,True,69247,True
152631,379,7367.0,2019-12-14 19:30:00,2019-12-14 19:40:00,2.0,True,True,69247,False
152632,379,14733.0,2019-12-14 19:40:00,2019-12-14 19:50:00,2.0,True,True,69247,False
152633,379,22100.0,2019-12-14 19:50:00,2019-12-14 20:00:00,0.0,True,True,69248,True
152634,379,0.0,2019-12-14 20:00:00,2019-12-14 20:10:00,0.0,True,False,69249,True
152635,379,0.0,2019-12-14 20:10:00,2019-12-14 20:20:00,0.0,True,False,69250,True
152636,379,0.0,2019-12-14 20:20:00,2019-12-14 20:30:00,3.0,True,True,69251,True
152637,379,0.0,2019-12-14 20:30:00,2019-12-14 20:40:00,3.0,True,True,69251,False
152638,379,0.0,2019-12-14 20:40:00,2019-12-14 20:50:00,3.0,True,True,69251,False


En la siguiente celda mostramos algunas filas donde se evidencia el llenado de valores nan entre periodos de tiempo mayores a 30min. Un ejemplo ocurre entre 2019-12-14 15:20:00 y 2019-12-14 16:00:00

In [283]:
Rain_aux[(Rain_aux['Gaps']==4)&(Rain_aux['Rain']!=0)]
Rain_aux.loc[152600:152620]

,Sta,Rain,T_0,T_f,Gaps,Gaps<3,Near,oldInd,old
152600,379,NaN,2019-12-14 14:20:00,2019-12-14 14:30:00,4.0,False,True,69233,False
152601,379,NaN,2019-12-14 14:30:00,2019-12-14 14:40:00,4.0,False,True,69233,False
152602,379,NaN,2019-12-14 14:40:00,2019-12-14 14:50:00,4.0,False,True,69233,False
152603,379,NaN,2019-12-14 14:50:00,2019-12-14 15:00:00,4.0,False,True,69233,False
152604,379,0.0,2019-12-14 15:00:00,2019-12-14 15:10:00,0.0,True,True,69234,True
152605,379,300.0,2019-12-14 15:10:00,2019-12-14 15:20:00,4.0,False,True,69235,True
152606,379,NaN,2019-12-14 15:20:00,2019-12-14 15:30:00,4.0,False,True,69235,False
152607,379,NaN,2019-12-14 15:30:00,2019-12-14 15:40:00,4.0,False,True,69235,False
152608,379,NaN,2019-12-14 15:40:00,2019-12-14 15:50:00,4.0,False,True,69235,False
152609,379,NaN,2019-12-14 15:50:00,2019-12-14 16:00:00,4.0,False,True,69235,False


En la siguiente celda realizamos una agrupación en periodos de tiempo de 30 minutos. Aquellos periodos de tiempo de 30 minutos con al menos un registro faltante se reemplaza por un NAN

In [ ]:
Rain_aux0 = Rain_aux.set_index('T_0').copy()
rain_30min = Rain_aux0['Rain'].resample('30min').apply(lambda x: np.nan if x.isna().any() else x.sum())
rain_30min = rain_30min.reset_index()

En la siguiente celda mostramos nuevamente algunas filas de interés. Por ejemplo, se realiza la agrupación de los registros entre 2019-12-14 19:30:00 y 2019-12-14 20:00:00

In [273]:
rain_30min[rain_30min['T_0'].dt.date == pd.to_datetime('2019-12-14').date()][-10:] #15:00

,T_0,Rain
50876,2019-12-14 19:00:00,0.0
50877,2019-12-14 19:30:00,44200.0
50878,2019-12-14 20:00:00,0.0
50879,2019-12-14 20:30:00,0.0
50880,2019-12-14 21:00:00,0.0
50881,2019-12-14 21:30:00,0.0
50882,2019-12-14 22:00:00,0.0
50883,2019-12-14 22:30:00,0.0
50884,2019-12-14 23:00:00,0.0
50885,2019-12-14 23:30:00,0.0


En la siguiente celda mostramos nuevamente algunas filas donde se evidencia la agrupación en periodos de 30 minutos en los casos de existencia de valores nan en dicho periodo. Un ejemplo ocurre entre 2019-12-14 15:20:00 y 2019-12-14 16:00:00

In [286]:
rain_30min[rain_30min['T_0'].dt.date == pd.to_datetime('2019-12-14').date()][-20:-10] #15:00

,T_0,Rain
50866,2019-12-14 14:00:00,NaN
50867,2019-12-14 14:30:00,NaN
50868,2019-12-14 15:00:00,NaN
50869,2019-12-14 15:30:00,NaN
50870,2019-12-14 16:00:00,0.0
50871,2019-12-14 16:30:00,0.0
50872,2019-12-14 17:00:00,0.0
50873,2019-12-14 17:30:00,0.0
50874,2019-12-14 18:00:00,0.0
50875,2019-12-14 18:30:00,0.0


Finalmente, reorganizamos las columnas y agregamos las columnas T_f y Sta.

In [288]:
rain_30min['Sta'] = Rain_aux['Sta'].values[0]
rain_30min['T_f'] = rain_30min['T_0'] + pd.Timedelta('30 minutes')
rain_30min = rain_30min[['Sta','Rain','T_f']]
rain_30min

,Sta,Rain,T_f
0,379,0.0,2017-01-18 21:30:00
1,379,0.0,2017-01-18 22:00:00
2,379,NaN,2017-01-18 22:30:00
3,379,NaN,2017-01-18 23:00:00
4,379,NaN,2017-01-18 23:30:00
...,...,...,...
135697,379,0.0,2024-10-15 22:00:00
135698,379,0.0,2024-10-15 22:30:00
135699,379,0.0,2024-10-15 23:00:00
135700,379,0.0,2024-10-15 23:30:00
